In [1]:
import pandas as pd
from gensim import corpora, models
from gensim.similarities import MatrixSimilarity
from gensim.utils import SaveLoad
from bs4 import BeautifulSoup
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re 
import pyLDAvis.gensim
from collections import Counter
from gensim.matutils import corpus2csc, sparse2full, corpus2dense
import gensim

In [2]:
SF = pd.read_csv('SF_jobs_new.csv')
SF.drop('Unnamed: 0',axis=1,inplace=True)
NYC = pd.read_csv('NYC_jobs_new.csv')
NYC.drop('Unnamed: 0',axis=1,inplace=True)
Seattle = pd.read_csv('Seattle_jobs_new.csv')
Seattle.drop('Unnamed: 0',axis=1,inplace=True)

In [4]:
def Preprocessing_line(line): # perform tokenization, select noun, Lemmertization etc on a line text
    rtext=[]    
    for w, tag in nltk.pos_tag(nltk.word_tokenize(line.lower())):  # Tokenization
        if tag in ['NN']:  # Keep only Nouns (for this special case)
            rtext.append(w)
    
    wordnet_lemmatizer = WordNetLemmatizer()  # Lemmertization
    rtext = [wordnet_lemmatizer.lemmatize(w) for w in rtext]
            
    stop_words = set(stopwords.words("english")) # Filter out any stop words
    rtext = [w for w in rtext if not w in stop_words]
    
    return rtext # return list of tokens

def text_cleaner(html,Oneline=True): # return a single line of cleaned text

    soup_obj = BeautifulSoup(html,"lxml") # Get the html from the site

    for script in soup_obj(["script", "style"]):
        script.extract() # Remove these two elements from the BS4 object
    
    text = soup_obj.body.get_text('\n') # Get the text from this

    lines = (line.strip() for line in text.splitlines()) # break into lines
#    lines = [line for i, line in enumerate(lines) if i>16]
#    for line in lines:
#        print line
    chunks = (phrase.strip() for line in lines for phrase in line.split("  ")) # break multi-headlines into a line each
    
    def chunk_space(chunk):
        chunk_out = chunk + ' ' # Need to fix spacing issue
        return chunk_out  

    text = ''.join(chunk_space(chunk) for chunk in chunks if chunk).encode('utf-8') # Get rid of all blank lines and ends of line

    # Now clean out all of the unicode junk (this line works great!!!)
    try:
        text = text.decode('unicode_escape').encode('ascii', 'ignore') # Need this as some websites aren't formatted
    except:                                                            # in a way that this works, can occasionally throw
        return                                                         # an exception

    text = re.sub("[^a-zA-Z+3#]"," ", text)  # Now get rid of any terms that aren't words (include 3 for d3.js)
      
    text = Preprocessing_line(text)
    
    return text

In [5]:
cleanDescription = []
for i in range(SF.shape[0]):
    cleanDescription.append(text_cleaner(SF.job_description[i]))
for i in range(NYC.shape[0]):
    cleanDescription.append(text_cleaner(NYC.job_description[i]))
for i in range(Seattle.shape[0]):
    cleanDescription.append(text_cleaner(Seattle.job_description[i]))

In [2]:
cleaneddoc = pd.read_pickle('cleaned_doc.txt')
dictionary = gensim.corpora.Dictionary(cleaneddoc)
corpus = [dictionary.doc2bow(text) for text in cleaneddoc]
tfidf = gensim.models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]


new_texts=[]
for i, doc in enumerate(corpus_tfidf):
    text = []
    for j, (_,tfidf) in enumerate(doc):
        if tfidf >= 0.25:
            text.append(cleaneddoc[i][j])
    new_texts.append(text)
   
cleanDescription = new_texts

In [10]:
def makeDict(myTweetList):
    """Create dictionary from list of tokenized documents"""
    return corpora.Dictionary(myTweetList)

def makeCorpus(myTweetList,myDict):
    """Create corpus from list of tokenized documents"""
    return [myDict.doc2bow(tweet) for tweet in myTweetList]

def createLDA(myCorpus, myDictionary,myTopics=10,myPasses=10,myIterations=50,myAlpha=0.001):
    """LDA model call function"""
    return models.LdaMulticore(myCorpus, id2word=myDictionary, num_topics=myTopics, passes=myPasses,
    iterations=myIterations,alpha=myAlpha)
    #return models.ldamodel.LdaModel(myCorpus, id2word=myDictionary, num_topics=myTopics, update_every=1, chunksize=100, passes=1,gamma_threshold=0.001, minimum_probability=0.01, alpha = 'asymmetric')

In [11]:
kagDict   = makeDict(cleanDescription)
kagCorpus = makeCorpus(cleanDescription, kagDict)
kagLda = createLDA(kagCorpus, kagDict)

In [12]:
pyLDAvis.enable_notebook()
ldaViz = pyLDAvis.gensim.prepare(kagLda, kagCorpus, kagDict)
ldaViz

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
2      11.333038        1       1 -0.036770  0.114048
7      11.045520        1       2 -0.271102 -0.076424
9      10.872657        1       3 -0.011344 -0.148116
6      10.856966        1       4  0.098774  0.003312
1      10.328550        1       5  0.017310 -0.004630
4      10.097187        1       6  0.102521 -0.060813
3       9.358283        1       7 -0.048605  0.209083
8       9.163666        1       8  0.042286  0.007402
0       8.542867        1       9  0.055366 -0.028071
5       8.401267        1      10  0.051564 -0.015790, topic_info=     Category        Freq            Term       Total  loglift  logprob
term                                                                   
2008  Default  225.000000       knowledge  225.000000  30.0000  30.0000
616   Default  151.000000            team  151.000000  29.0000  29.0000
249   Default  103.000000         project  103.000000  28.0000  28.0000
199   Default   91.000000          resume   91.000000  27.0000  27.0000
2222  Default  151.000000        software  151.000000  26.0000  26.0000
1093  Default  100.000000   communication  100.000000  25.0000  25.0000
1553  Default   72.000000        database   72.000000  24.0000  24.0000
1539  Default  201.000000     development  201.000000  23.0000  23.0000
93    Default   87.000000            work   87.000000  22.0000  22.0000
1239  Default   74.000000         science   74.000000  21.0000  21.0000
708   Default   64.000000         product   64.000000  20.0000  20.0000
519   Default  100.000000         support  100.000000  19.0000  19.0000
181   Default   69.000000         network   69.000000  18.0000  18.0000
1942  Default   80.000000          system   80.000000  17.0000  17.0000
2149  Default   88.000000        position   88.000000  16.0000  16.0000
228   Default   50.000000         service   50.000000  15.0000  15.0000
599   Default  110.000000     environment  110.000000  14.0000  14.0000
572   Default   91.000000             sql   91.000000  13.0000  13.0000
1924  Default   77.000000          please   77.000000  12.0000  12.0000
1841  Default   66.000000         process   66.000000  11.0000  11.0000
1232  Default   51.000000         company   51.000000  10.0000  10.0000
836   Default   71.000000        computer   71.000000   9.0000   9.0000
229   Default   47.000000     opportunity   47.000000   8.0000   8.0000
153   Default   35.000000      production   35.000000   7.0000   7.0000
268   Default   79.000000          client   79.000000   6.0000   6.0000
1498  Default   82.000000            java   82.000000   5.0000   5.0000
2162  Default   59.000000             web   59.000000   4.0000   4.0000
1703  Default   83.000000     application   83.000000   3.0000   3.0000
565   Default   58.000000            role   58.000000   2.0000   2.0000
931   Default   52.000000  infrastructure   52.000000   1.0000   1.0000
...       ...         ...             ...         ...      ...      ...
931   Topic10   26.735430  infrastructure   52.443138   1.8030  -3.8206
1697  Topic10   30.031423     engineering   62.641438   1.7416  -3.7044
2272  Topic10    6.921834             inc   12.008742   1.9258  -5.1720
1498  Topic10   36.543256            java   82.135833   1.6669  -3.5081
119   Topic10    7.604626     methodology   13.624083   1.8937  -5.0779
737   Topic10    4.919023      consulting    8.510702   1.9286  -5.5135
861   Topic10   23.868153     performance   58.064617   1.5878  -3.9341
111   Topic10   19.066120             end   46.022129   1.5956  -4.1587
1387  Topic10   11.091450               b   23.932088   1.7077  -4.7005
805   Topic10    8.665228        language   18.000174   1.7457  -4.9473
1175  Topic10    4.289307             fax    7.659023   1.8970  -5.6505
1235  Topic10    7.619669        hardware   17.962170   1.6193  -5.0759
2261  Topic10    4.272937    distribution    7.703871   1.8874  -5.65